In [1]:
import os
import json
import math
import numpy as np
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import FashionMNIST
from torchvision import transforms

/var/folders/_8/8qs28kr50_v5rng3_4jhcdfr0000gn/T/ipykernel_25980/1331318064.py:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [2]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cpu


In [3]:
# Transformations applied on each image => only make them a tensor
transform = transforms.Compose([transforms.ToTensor()])

# Loading the training dataset. We need to split it into a training and validation part
complete_set = torchvision.datasets.ImageFolder("trafic_32", transform=transform)


train_set, test_set = data.dataset.random_split(complete_set, [0.8, 0.2], generator=torch.Generator().manual_seed(42))

# We define a set of data loaders that we can use for various purposes later.
train_loader = data.DataLoader(train_set, batch_size=256, shuffle=True, drop_last=True, pin_memory=True, num_workers=8)
test_loader = data.DataLoader(test_set, batch_size=256, shuffle=False, drop_last=True, num_workers=8)

def get_train_images(num):
    return torch.stack([test_set[i][0] for i in range(10,10+num)], dim=0)

In [4]:
train_set[0]

(tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0039, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0078],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0118, 0.0118, 0.0000,  ..., 0.0000, 0.0039, 0.0000],
          [0.0118, 0.0118, 0.0039,  ..., 0.0039, 0.0078, 0.0039],
          [0.0157, 0.0118, 0.0039,  ..., 0.0039, 0.0157, 0.0157]],
 
         [[0.0000, 0.0000, 0.0039,  ..., 0.0078, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0039, 0.0078],
          [0.0000, 0.0000, 0.0000,  ..., 0.0078, 0.0039, 0.0078],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0039, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0078, 0.0157, 0.0157,  ..., 0.0157, 0.0000, 0.0000],
          [0.0078, 0.0078, 0.0157,  ..., 0.0039, 0.0000, 0.0078],
          [0.0078, 0.0078, 0.0157,  ...,

In [5]:
train_set[0][0].shape

torch.Size([3, 32, 32])

In [6]:
class Encoder(nn.Module):
    def __init__(self, feature_dim, z_dim):
        super(Encoder, self).__init__()

        self.activation = nn.LeakyReLU(0.2)

        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 6, 7),
            #26*26*3
            self.activation,
            nn.MaxPool2d(2),
            #13*13*6

            nn.Conv2d(6, 12, 3),
            #11*11*12
            self.activation,
            nn.MaxPool2d(2),
            #5*5*12

            nn.Conv2d(12, 12, 3),
            #3*3*12=108
            self.activation,


        )

        self.linear_layer = nn.Sequential(
            nn.Linear(3*3*12, feature_dim),
            self.activation,
            nn.Linear(feature_dim, feature_dim),
            self.activation
        )

        self.fc_mean  = nn.Linear(feature_dim, z_dim)
        self.fc_var   = nn.Linear (feature_dim, z_dim)

        self.training = True

    def forward(self, x):
        x       = self.conv_layer(x)
        x       = torch.flatten(x, 1)
        x       = self.linear_layer(x)
        mean    = self.fc_mean(x)
        log_var = self.fc_var(x)
        # encoder produces mean and log of variance
        #(i.e., parateters of simple tractable normal distribution "q"

        return mean, log_var

In [7]:
class Decoder(nn.Module):
    def __init__(self, z_dim, feature_dim):
        super(Decoder, self).__init__()

        self.activation = nn.LeakyReLU(0.2)

        self.linear_layer = nn.Sequential(
            nn.Linear(z_dim, feature_dim),
            self.activation,
            nn.Linear(feature_dim, 3*3*12),
            self.activation,

        )
        self.unflatten =  nn.Unflatten(dim=1, unflattened_size=(12, 3, 3))


        self.deconv_layer = nn.Sequential(
            nn.ConvTranspose2d(12, 12, 3),
            self.activation,
            nn.MaxUnpool2d(2),

            nn.ConvTranspose2d(12, 6, 3),
            self.activation,
            nn.MaxUnpool2d(2),

            nn.ConvTranspose2d(6, 3, 7)
        )


    def forward(self, x):
        x = self.linear_layer(x)
        x = self.unflatten(x)

        x = self.deconv_layer(x)
        x = torch.sigmoid(x)

        return x


In [8]:
class VAE(nn.Module):
    def __init__(self,feature_dim, z_dim):
        super(VAE, self).__init__()
        self.z_dim = z_dim
        self.encoder = Encoder(feature_dim=feature_dim, z_dim=z_dim)
        self.decoder = Decoder(z_dim=z_dim, feature_dim = feature_dim)


    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(device)
        z = mean + var * epsilon
        return z


    def forward(self, x):
        mean, log_var = self.encoder(x)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # takes exponential function (log var -> var)
        x_hat = self.decoder(z)
        return x_hat, mean, log_var

In [9]:
vae = VAE(feature_dim=64, z_dim=16)

In [10]:
def vae_loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    KLD = -0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())
    return reproduction_loss + KLD

In [11]:
optimizer = optim.Adam(vae.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.99)

In [12]:
num_epochs = 30
for n in range(num_epochs):
    losses_epoch = []
    for x, _ in iter(train_loader):
        x = x.to(device)
        out, means, log_var = vae(x)
        loss = vae_loss_function(x, out, means, log_var)
        losses_epoch.append(loss.item())
        loss.backward()               # backward pass (compute parameter updates)
        optimizer.step()              # make the updates for each parameter
        optimizer.zero_grad()
    L1_list = []
#     if n % 10 == 0:
    for x, _ in iter(test_loader):
        x  = x.to(device)
        out, _, _ = vae(x)
        L1_list.append(torch.mean(torch.abs(out-x)).item())
    print(f"Epoch {n} loss {np.mean(np.array(losses_epoch))}, test L1 = {np.mean(L1_list)}")
    scheduler.step()

TypeError: forward() missing 1 required positional argument: 'indices'

In [ ]:
def generate_images(model, n_imgs, device):
    # Generate images
    model.eval()
    with torch.no_grad():
        generated_imgs = model.decoder(torch.randn([n_imgs, model.z_dim]).to(device))
    generated_imgs = generated_imgs.cpu()

    grid = torchvision.utils.make_grid(generated_imgs, nrow=4, normalize=False, range=(-1,1))
    grid = grid.permute(1, 2, 0)
    if len(input_imgs) == 4:
        plt.figure(figsize=(10,10))
    else:
        plt.figure(figsize=(15,10))
    plt.title(f"Generations")
    plt.imshow(grid)
    plt.axis('off')
    plt.show()

In [ ]:
generate_images(vae, 16 , device)